# Hymba Ablation Study - Tiny Shakespeare

## 목표
다양한 아키텍처 구성의 성능을 비교하여 각 컴포넌트의 기여도를 분석합니다.

### 테스트 구성
1. **Mamba-only**: SSM 기반 시퀀스 모델링
2. **Transformer-only**: 표준 어텐션 기반
3. **Hybrid (Hymba)**: Attention + Mamba 혼합
   - Global Attention: 첫/중간/마지막 레이어
   - Local Attention (SWA): 나머지 레이어
   - Meta Tokens: 64개
   - KV-Cache 공유

### 평가 메트릭
- 학습 Loss & Perplexity
- 검증 Loss & Perplexity
- 학습 속도 (tokens/sec)
- 추론 속도 (tokens/sec)
- KV-Cache 메모리 절감
- 생성 품질

In [ ]:
import sys
import os
import warnings
sys.path.append('./backbone')

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
warnings.filterwarnings('ignore')

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm

import importlib
if 'hymba' in sys.modules:
    importlib.reload(sys.modules['hymba'])

from hymba import Hymba, HymbaConfig, ArchType, AttentionType

from datasets import load_dataset
from torch.utils.data import DataLoader, TensorDataset, random_split

from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.trainers import UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Sequence as NormSeq, NFKC, Lowercase

RESULTS_DIR = './results'
os.makedirs(RESULTS_DIR, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 150
sns.set_style('whitegrid')

## 1. 데이터 준비

In [ ]:
print('데이터 로딩 중...')
ds = load_dataset('karpathy/tiny_shakespeare')
text = '\n\n'.join(ds['train']['text'])
print(f'전체 텍스트 길이: {len(text):,} 문자')

In [ ]:
print('토크나이저 학습 중...')
vocab_size = 4000

tk = Tokenizer(Unigram())
tk.normalizer = NormSeq([NFKC(), Lowercase()])
tk.pre_tokenizer = Whitespace()
trainer = UnigramTrainer(vocab_size=vocab_size, special_tokens=['<|unk|>'], unk_token='<|unk|>')
tk.train_from_iterator([text], trainer=trainer)

class TokenizerWrap:
    def __init__(self, tk):
        self.tk = tk
    def encode(self, s):
        return self.tk.encode(s).ids
    def decode(self, ids):
        return self.tk.decode(ids)
    @property
    def vocab_size(self):
        return self.tk.get_vocab_size()

tokenizer = TokenizerWrap(tk)
print(f'어휘 크기: {tokenizer.vocab_size}')

In [ ]:
def make_dataset(tok, text, seq_len=256):
    ids = np.array(tok.encode(text), dtype=np.int64)
    x, y = ids[:-1], ids[1:]
    n = (len(y) // seq_len) * seq_len
    X = torch.tensor(x[:n].reshape(-1, seq_len))
    Y = torch.tensor(y[:n].reshape(-1, seq_len))
    return TensorDataset(X, Y)

seq_len = 256
batch_size = 16

ds_full = make_dataset(tokenizer, text, seq_len)
tr_len = int(0.9 * len(ds_full))
va_len = len(ds_full) - tr_len
train_ds, val_ds = random_split(ds_full, [tr_len, va_len])

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

print(f'학습 배치: {len(train_dl)}')
print(f'검증 배치: {len(val_dl)}')

## 2. 모델 구성 정의

In [ ]:
# 공정한 비교를 위해 파라미터 수를 ~50M으로 맞춤
# 각 아키텍처의 특성에 맞게 d_model 조정

common_params = {
    'vocab_size': vocab_size,
    'swa_window': 128,
    'dropout': 0.1,
    'seq_len': seq_len,
}

# 파라미터 수 목표: ~50M
configs = {
    # Mamba-only: d_model을 조정 (Mamba가 expand=2로 파라미터가 많음)
    'Mamba-only': HymbaConfig(
        **common_params,
        d_model=448,
        n_layers=12,
        n_heads=8,
        n_kv_heads=4,
        arch_type=ArchType.MAMBA_ONLY,
        mamba_heads_per_layer=1,
        use_meta_tokens=False,
        use_kv_sharing=False,
    ),
    # Transformer-only: 기준 모델 (49M)
    'Transformer-only': HymbaConfig(
        **common_params,
        d_model=512,
        n_layers=12,
        n_heads=8,
        n_kv_heads=4,
        arch_type=ArchType.TRANSFORMER_ONLY,
        global_attn_indices=list(range(12)),  # 모든 레이어 Global
        use_meta_tokens=False,
        use_kv_sharing=False,
    ),
    # Hybrid (1:1): Attn + Mamba 각 1개씩
    'Hybrid (1:1)': HymbaConfig(
        **common_params,
        d_model=384,
        n_layers=12,
        n_heads=8,
        n_kv_heads=4,
        arch_type=ArchType.HYBRID,
        mamba_heads_per_layer=1,
        global_attn_indices=[0, 5, 11],  # 첫/중간/마지막 Global
        use_meta_tokens=True,
        num_meta_tokens=64,
        use_kv_sharing=True,
    ),
    # Hybrid (2:1): Mamba 비중 높임
    'Hybrid (2:1)': HymbaConfig(
        **common_params,
        d_model=352,
        n_layers=12,
        n_heads=8,
        n_kv_heads=4,
        arch_type=ArchType.HYBRID,
        mamba_heads_per_layer=2,
        global_attn_indices=[0, 5, 11],
        use_meta_tokens=True,
        num_meta_tokens=64,
        use_kv_sharing=True,
    ),
}

# 파라미터 수 검증
print('=== 파라미터 수 검증 (목표: ~50M) ===')
for name, cfg in configs.items():
    model = Hymba(cfg)
    params = model.count_parameters()['total']
    layer_configs = cfg.get_layer_configs()
    total_attn = sum(c[0] for c in layer_configs)
    total_mamba = sum(c[1] for c in layer_configs)
    print(f'{name:20s}: {params/1e6:5.2f}M params | d={cfg.d_model:3d} | Attn={total_attn:3d} | Mamba={total_mamba:3d}')
    del model
    torch.cuda.empty_cache()
print('='*70)

## 3. 학습 함수

In [ ]:
def train_model(model, train_dl, val_dl, epochs=30, lr=3e-4, warmup_steps=200, eval_interval=32, device='cuda'):
    """모델 학습 함수"""
    model = model.to(device).train()
    
    # Optimizer: AdamW with weight decay
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.95), weight_decay=0.1)
    total_steps = epochs * len(train_dl)
    
    # Learning rate schedule: warmup + cosine decay
    def lr_schedule(step):
        if step < warmup_steps:
            return step / warmup_steps
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
        return 0.5 * (1 + np.cos(np.pi * progress))
    
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_schedule)
    
    # Mixed precision training
    use_amp = device == 'cuda' and torch.cuda.is_bf16_supported()
    scaler = torch.amp.GradScaler('cuda') if use_amp else None
    
    history = {'train_loss': [], 'val_loss': [], 'val_ppl': [], 'lr': [], 'step': []}
    step, t0, best_val_loss = 0, time.time(), float('inf')
    
    for epoch in range(epochs):
        model.train()
        epoch_loss, epoch_tokens = 0.0, 0
        pbar = tqdm(train_dl, desc=f'Epoch {epoch+1}/{epochs}', leave=False)
        
        for xb, yb in pbar:
            xb, yb = xb.to(device), yb.to(device)
            
            if use_amp:
                with torch.amp.autocast('cuda', dtype=torch.bfloat16):
                    out = model(xb, targets=yb)
                    loss = out['loss']
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(xb, targets=yb)
                loss = out['loss']
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            epoch_loss += loss.item() * xb.numel()
            epoch_tokens += xb.numel()
            step += 1
            
            # Evaluation
            if step % eval_interval == 0:
                model.eval()
                val_loss, val_tokens = 0.0, 0
                with torch.no_grad():
                    for vxb, vyb in val_dl:
                        vxb, vyb = vxb.to(device), vyb.to(device)
                        if use_amp:
                            with torch.amp.autocast('cuda', dtype=torch.bfloat16):
                                vout = model(vxb, targets=vyb)
                        else:
                            vout = model(vxb, targets=vyb)
                        val_loss += vout['loss'].item() * vxb.numel()
                        val_tokens += vxb.numel()
                
                val_loss /= val_tokens
                train_loss = epoch_loss / epoch_tokens
                history['train_loss'].append(train_loss)
                history['val_loss'].append(val_loss)
                history['val_ppl'].append(np.exp(val_loss))
                history['lr'].append(scheduler.get_last_lr()[0])
                history['step'].append(step)
                best_val_loss = min(best_val_loss, val_loss)
                pbar.set_postfix({'loss': f'{train_loss:.3f}', 'val': f'{val_loss:.3f}', 'ppl': f'{np.exp(val_loss):.1f}'})
                model.train()
    
    elapsed = time.time() - t0
    tps = int(epochs * len(train_dl) * batch_size * seq_len / elapsed)
    
    return {
        'best_val_loss': best_val_loss,
        'best_val_ppl': np.exp(best_val_loss),
        'final_val_ppl': history['val_ppl'][-1] if history['val_ppl'] else np.exp(best_val_loss),
        'train_tps': tps,
        'time_min': elapsed / 60,
        'history': history,
    }

In [ ]:
def benchmark_inference(model, tokenizer, prompt='ROMEO:', max_tokens=100, n_runs=3, device='cuda'):
    model = model.to(device).eval()
    prompt_tensor = torch.tensor([tokenizer.encode(prompt)]).to(device)
    times = []
    for _ in range(n_runs):
        torch.cuda.synchronize() if device == 'cuda' else None
        t0 = time.time()
        with torch.no_grad():
            _ = model.generate(prompt_tensor, max_new_tokens=max_tokens, temperature=1.0)
        torch.cuda.synchronize() if device == 'cuda' else None
        times.append(time.time() - t0)
    return {'tokens_per_sec': max_tokens / np.mean(times)}

## 4. 모델 학습

In [ ]:
results = {}

for name, cfg in configs.items():
    print(f"\n{'='*60}")
    print(f"Model: {name}")
    print(f"{'='*60}")
    
    model = Hymba(cfg)
    params = model.count_parameters()
    print(f'Parameters: {params["total"]/1e6:.2f}M | d_model: {cfg.d_model}')
    
    # KV cache reduction (Hybrid에서만 의미 있음)
    kv_reduction = 1.0
    if cfg.arch_type != ArchType.MAMBA_ONLY:
        kv_info = model.get_kv_sharing_info()
        kv_reduction = kv_info['reduction']
        if cfg.use_kv_sharing:
            print(f'KV Reduction: {kv_reduction:.2f}x')
    
    # 학습
    train_results = train_model(model, train_dl, val_dl, epochs=30, lr=3e-4, device=device)
    
    # 추론 벤치마크
    infer_results = benchmark_inference(model, tokenizer, device=device)
    
    # 샘플 생성
    samples = []
    for prompt in ['ROMEO:', 'First Citizen:', 'KING:']:
        prompt_tensor = torch.tensor([tokenizer.encode(prompt)]).to(device)
        with torch.no_grad():
            gen = model.generate(prompt_tensor, max_new_tokens=80, temperature=0.8, top_k=40)
        samples.append(tokenizer.decode(gen[0].cpu().tolist()))
    
    results[name] = {
        **train_results,
        **infer_results,
        'params': params['total'],
        'd_model': cfg.d_model,
        'kv_reduction': kv_reduction,
        'samples': samples,
    }
    
    print(f'Best PPL: {train_results["best_val_ppl"]:.2f}')
    print(f'Inference: {infer_results["tokens_per_sec"]:.1f} tok/s')
    
    # 메모리 정리
    del model
    torch.cuda.empty_cache()

## 5. 결과 요약

In [ ]:
# 결과 DataFrame 생성
df = pd.DataFrame([{
    'Model': name,
    'd_model': r['d_model'],
    'Params (M)': r['params'] / 1e6,
    'Best PPL': r['best_val_ppl'],
    'Infer TPS': r['tokens_per_sec'],
    'KV Red.': r['kv_reduction'],
    'Time (min)': r['time_min'],
} for name, r in results.items()])

print('\n' + '='*90)
print('Results Summary (Fair Comparison with Similar Parameter Counts)')
print('='*90)
print(df.to_string(index=False))
print('='*90)

# 가장 좋은 모델 하이라이트
best_ppl_model = df.loc[df['Best PPL'].idxmin(), 'Model']
best_ppl_val = df['Best PPL'].min()
print(f'\n*** Best PPL: {best_ppl_model} ({best_ppl_val:.2f}) ***')

## 6. 시각화

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Performance (PPL) - 낮을수록 좋음
ax = axes[0, 0]
colors = ['coral' if 'Hybrid' in m else 'skyblue' for m in df['Model']]
bars = ax.barh(df['Model'], df['Best PPL'], color=colors)
ax.set_xlabel('Best Val PPL (lower is better)')
ax.set_title('Model Performance')
ax.invert_yaxis()
# 최고 성능 표시
best_idx = df['Best PPL'].idxmin()
bars[best_idx].set_color('green')
bars[best_idx].set_edgecolor('darkgreen')
bars[best_idx].set_linewidth(2)

# 2. Parameter Count - 비슷해야 공정한 비교
ax = axes[0, 1]
ax.barh(df['Model'], df['Params (M)'], color='lightyellow', edgecolor='orange')
ax.set_xlabel('Parameters (M)')
ax.set_title('Model Size (Target: ~50M)')
ax.axvline(x=50, color='red', linestyle='--', alpha=0.7, label='Target')
ax.invert_yaxis()
ax.legend()

# 3. Inference Speed
ax = axes[0, 2]
ax.barh(df['Model'], df['Infer TPS'], color='lightgreen')
ax.set_xlabel('Inference Speed (tok/s)')
ax.set_title('Inference Speed')
ax.invert_yaxis()

# 4. Training Curves
ax = axes[1, 0]
for name, r in results.items():
    ax.plot(r['history']['step'], r['history']['val_ppl'], label=name, linewidth=2)
ax.set_xlabel('Steps')
ax.set_ylabel('Val PPL')
ax.set_title('Training Curves')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 5. KV Cache Reduction
ax = axes[1, 1]
kv_colors = ['gray' if x == 1.0 else 'orange' for x in df['KV Red.']]
ax.barh(df['Model'], df['KV Red.'], color=kv_colors)
ax.set_xlabel('KV Reduction')
ax.set_title('Memory Efficiency (higher is better)')
ax.invert_yaxis()

# 6. Training Time
ax = axes[1, 2]
ax.barh(df['Model'], df['Time (min)'], color='plum')
ax.set_xlabel('Training Time (min)')
ax.set_title('Training Duration')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/ablation_results.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. 생성 품질

In [ ]:
prompts = ['ROMEO:', 'First Citizen:', 'KING:']
for i, prompt in enumerate(prompts):
    print(f"\n{'='*60}\nPrompt: {prompt}\n{'='*60}")
    for name, r in results.items():
        print(f'\n[{name}]')
        print(r['samples'][i])

## 8. 결론

In [ ]:
print('='*70)
print('Key Findings (Fair Comparison with ~50M Parameters)')
print('='*70)

# 1. Best Performance
best_ppl = df.loc[df['Best PPL'].idxmin()]
print(f'1. Best Performance: {best_ppl["Model"]} (PPL: {best_ppl["Best PPL"]:.2f})')

# 2. Fastest Inference
best_infer = df.loc[df['Infer TPS'].idxmax()]
print(f'2. Fastest Inference: {best_infer["Model"]} ({best_infer["Infer TPS"]:.1f} tok/s)')

# 3. Best Memory Efficiency
best_kv = df.loc[df['KV Red.'].idxmax()]
if best_kv['KV Red.'] > 1.0:
    print(f'3. Best Memory Efficiency: {best_kv["Model"]} ({best_kv["KV Red."]:.2f}x KV reduction)')
else:
    print(f'3. Memory Efficiency: Hybrid models achieve KV cache reduction via sharing')

print('\n' + '='*70)
print('Conclusions')
print('='*70)
print("""
- Mamba-only: Fast inference due to O(n) complexity, no KV cache needed
- Transformer-only: Strong performance with global attention
- Hybrid (Hymba): 
  * Combines Mamba's efficiency with Attention's expressiveness
  * KV cache sharing reduces memory footprint
  * Meta tokens help with attention sink problem
  * Global attention at key layers (first/middle/last)
  * Local (SWA) attention elsewhere for efficiency
""")

# Hybrid vs Single-arch 비교
hybrid_models = df[df['Model'].str.contains('Hybrid')]
single_models = df[~df['Model'].str.contains('Hybrid')]

if not hybrid_models.empty and not single_models.empty:
    best_hybrid_ppl = hybrid_models['Best PPL'].min()
    best_single_ppl = single_models['Best PPL'].min()
    
    if best_hybrid_ppl < best_single_ppl:
        improvement = (best_single_ppl - best_hybrid_ppl) / best_single_ppl * 100
        print(f'Hybrid achieves {improvement:.1f}% better PPL than best single-architecture model')

## 9. 결과 저장

In [ ]:
df.to_csv(f'{RESULTS_DIR}/ablation_results.csv', index=False)
print(f'Saved: {RESULTS_DIR}/ablation_results.csv')

with open(f'{RESULTS_DIR}/generation_samples.txt', 'w') as f:
    for i, prompt in enumerate(['ROMEO:', 'First Citizen:', 'KING:']):
        f.write(f'Prompt: {prompt}\n')
        for name, r in results.items():
            f.write(f'[{name}]\n{r["samples"][i]}\n\n')
print(f'Saved: {RESULTS_DIR}/generation_samples.txt')